In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dset

convnext_tiny = models.convnext_tiny(pretrained=True)

In [2]:
import os
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

TrainImagePath = "./train"
DevImagePath = "./test"
TrainImageNum = len(os.listdir(TrainImagePath))
DevImageNum = len(os.listdir(DevImagePath))
print("訓練資料筆數為{0}".format(TrainImageNum))
print("驗證資料筆數為{0}".format(DevImageNum))

normedWeights = [0.9492872602433018, 0.8414185138194392, 0.975234470234097, 0.8797795855412095, 0.9756573873672164, 0.9048187675697191, 0.9412269572356146, 0.8691071472995497, 0.9003408214543374,
 0.9791402343458467, 0.901087145806901, 0.9723486827375177, 0.9812796974898624, 0.9292733288553872]
normedWeights = torch.FloatTensor(normedWeights).to(device)

GPU State: cuda:0
訓練資料筆數為64323
驗證資料筆數為16071


In [3]:
class CropDataset(Dataset):
    """Crop dataset."""

    def __init__(self, root_dir, label_dict, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.label_dict = label_dict
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(os.listdir(self.root_dir))
    
    def __getitem__(self, idx):
        img_name = os.listdir(self.root_dir)[idx]
        image_path = os.path.join(self.root_dir, img_name)
        image = Image.open(image_path).convert('RGB')
        
        label = self.label_dict[img_name.split("_")[0]]
        if self.transform:
            image = self.transform(image)
        return image, label 

In [4]:
from torch.utils.data import WeightedRandomSampler 

featuretransform = transforms.Compose([transforms.Resize(255), 
                                       transforms.CenterCrop(224),  
#                                        transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(), 
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) 

label_dict = {"banana":0, "bareland":1, "carrot":2, "corn":3, "dragonfruit":4, "garlic":5, "guava":6, "peanut":7, 
                 "pineapple":8, "pumpkin":9, "rice":10, "sugarcane":11, "tomato":12, "soybean":13}

# sampler = WeightedRandomSampler(weight_list, TrainImageNum, replacement=True)

mongoDataset = CropDataset(TrainImagePath, label_dict, featuretransform)
valDataset = CropDataset(DevImagePath, label_dict, featuretransform)

# imgLoader = torch.utils.data.DataLoader(
#          mongoDataset,batch_size= 128, shuffle= False, num_workers= 10,sampler=sampler)

imgLoader = torch.utils.data.DataLoader(
         mongoDataset,batch_size= 256, shuffle= False, num_workers= 10)

valLoader = torch.utils.data.DataLoader(
         valDataset,batch_size= 256, shuffle= False, num_workers= 10)

In [5]:
#fine tune the last classifier layer from 1000 to 15 dim(num. of classifier).
convnext_tiny.classifier[2] = nn.Linear(768,14)
convnext_tiny = convnext_tiny.cuda() #use GPU

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [6]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss(weight=normedWeights)
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnext_tiny.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#scheduler = ReduceLROnPlateau(optimizer, 'min')
model = convnext_tiny

In [7]:
# Record Training Information
total_step = len(imgLoader)
loss_list, acc_list, vacc_list = [], [], []
best_acc = 0
num_epochs = 20

for epoch in range(num_epochs):
    # Training Stage
    model.train()
    correct, total = 0, 0
    for i, (images, labels) in enumerate(imgLoader):
        images = images.to(device)
        labels = labels.to(device, dtype=torch.long)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            loss_list.append(loss.item())
    print('Training Accuracy: {} %'.format(100 * correct / total))
    acc_list.append(100 * correct / total)
    
    # Validation Stage
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in valLoader:
            images = images.to(device)
            labels = labels.to(device, dtype=torch.long)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Save model checkpoint
    vacc = 100*correct/total
    print('Validation Accuracy on the 16070 validation images: {} %'.format(vacc))
    if vacc > best_acc:
        best_acc = vacc
        torch.save(model.state_dict(), 'new_convnext_all_calss_epoch_20_weight.ckpt')
        print("Save Checkpoint!!")
    print()
    vacc_list.append(100 * correct / total)

Epoch [1/20], Step [100/252], Loss: 0.8647
Epoch [1/20], Step [200/252], Loss: 0.3597
Training Accuracy: 70.54708269203861 %
Validation Accuracy on the 16070 validation images: 89.26638043681164 %
Save Checkpoint!!

Epoch [2/20], Step [100/252], Loss: 0.2420
Epoch [2/20], Step [200/252], Loss: 0.0941
Training Accuracy: 93.00872160813395 %
Validation Accuracy on the 16070 validation images: 93.91450438678365 %
Save Checkpoint!!

Epoch [3/20], Step [100/252], Loss: 0.1877
Epoch [3/20], Step [200/252], Loss: 0.1474
Training Accuracy: 95.15569858371033 %
Validation Accuracy on the 16070 validation images: 91.7615580859934 %

Epoch [4/20], Step [100/252], Loss: 0.1262
Epoch [4/20], Step [200/252], Loss: 0.0941
Training Accuracy: 96.29992382196104 %
Validation Accuracy on the 16070 validation images: 95.0967581357725 %
Save Checkpoint!!

Epoch [5/20], Step [100/252], Loss: 0.0936
Epoch [5/20], Step [200/252], Loss: 0.1131
Training Accuracy: 96.91556674906332 %
Validation Accuracy on the 1607